<a href="https://colab.research.google.com/github/banbedangki/Clarity_1/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

// CLick Edit->Notebook setting

In [23]:
!pip uninstall -y tensorflow
# install TensorFlow-2.0 with GPU support
!pip install tensorflow-gpu

Uninstalling tensorflow-2.1.0rc1:
  Successfully uninstalled tensorflow-2.1.0rc1
  Using cached https://files.pythonhosted.org/packages/fc/08/8b927337b7019c374719145d1dceba21a8bb909b93b1ad6f8fb7d22c1ca1/tensorflow_estimator-2.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/76/54/99b9d5d52d5cb732f099baaaf7740403e83fe6b0cedde940fabd2b13d75a/tensorboard-2.0.2-py3-none-any.whl
  Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensorboard-2.1.0


In [24]:
# Load tensortflow 2
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass

import tensorflow as tf

print(tf.__version__)
tf.executing_eagerly()

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2.x  #gpu`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
2.1.0-rc1


True

In [25]:
import os
import tensorflow_datasets as tfds #https://www.tensorflow.org/datasets/catalog/overview

print(tf.__version__)

2.1.0-rc1


In [0]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

In [27]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [28]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [29]:
tf.test.is_gpu_available ()

True

In [0]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [0]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  # TODO: scaling value of each pixel here:
  image = image/255
  return image, label

In [32]:
print(mnist_train)

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>


In [0]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [34]:
print(train_dataset)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>


In [0]:
# TODO: set suitable value to below variables
HIDDEN_LAYER_SIZE = 128 #hyperparameter
OUTPUT_SIZE = 10

with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28,28,1)),
      tf.keras.layers.MaxPooling2D(), #2,2

      tf.keras.layers.Flatten(),
      
      # TODO: fill suitable activations
      tf.keras.layers.Dense(HIDDEN_LAYER_SIZE, activation='relu'),
      tf.keras.layers.Dense(OUTPUT_SIZE, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy', #https://keras.io/losses/
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [0]:
# TODO: set suitable value to below variables
hidden_layer_size = 128
output_size = 10

with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      
      tf.keras.layers.MaxPooling2D(),

      tf.keras.layers.Flatten(),        # Flatten() cat thanh cac vector don
      
      # TODO: fill suitable activations
      tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(output_size, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [0]:
# Define the checkpoint directory to store the checkpoints

checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [0]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
]

In [39]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

Epoch 1/12
938/938 [==============================] - 16s 17ms/step - loss: 0.1521 - accuracy: 0.9544
Epoch 2/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0448 - accuracy: 0.9860
Epoch 3/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0299 - accuracy: 0.9910
Epoch 4/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0226 - accuracy: 0.9931
Epoch 5/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0160 - accuracy: 0.9951
Epoch 6/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0129 - accuracy: 0.9957
Epoch 7/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0105 - accuracy: 0.9967
Epoch 8/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0074 - accuracy: 0.9976
Epoch 9/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0070 - accuracy: 0.9976
Epoch 10/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0078 - accuracy: 0.99

In [18]:
# check the checkpoint directory
!ls {checkpoint_dir}

checkpoint		     ckpt_4.data-00000-of-00002
ckpt_10.data-00000-of-00002  ckpt_4.data-00001-of-00002
ckpt_10.data-00001-of-00002  ckpt_4.index
ckpt_10.index		     ckpt_5.data-00000-of-00002
ckpt_11.data-00000-of-00002  ckpt_5.data-00001-of-00002
ckpt_11.data-00001-of-00002  ckpt_5.index
ckpt_11.index		     ckpt_6.data-00000-of-00002
ckpt_12.data-00000-of-00002  ckpt_6.data-00001-of-00002
ckpt_12.data-00001-of-00002  ckpt_6.index
ckpt_12.index		     ckpt_7.data-00000-of-00002
ckpt_1.data-00000-of-00002   ckpt_7.data-00001-of-00002
ckpt_1.data-00001-of-00002   ckpt_7.index
ckpt_1.index		     ckpt_8.data-00000-of-00002
ckpt_2.data-00000-of-00002   ckpt_8.data-00001-of-00002
ckpt_2.data-00001-of-00002   ckpt_8.index
ckpt_2.index		     ckpt_9.data-00000-of-00002
ckpt_3.data-00000-of-00002   ckpt_9.data-00001-of-00002
ckpt_3.data-00001-of-00002   ckpt_9.index
ckpt_3.index


In [40]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

    157/Unknown - 3s 20ms/step - loss: 0.0439 - accuracy: 0.9901Eval loss: 0.043941488650428365, Eval Accuracy: 0.9901000261306763


In [0]:
path = 'saved_model/'

In [42]:
model.save(path, save_format='tf')

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [43]:
unreplicated_model = tf.keras.models.load_model(path)

# unreplicated_model.fit (data)

unreplicated_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

    157/Unknown - 2s 13ms/step - loss: 0.0439 - accuracy: 0.9901Eval loss: 0.04394236892377211, Eval Accuracy: 0.9901000261306763


In [44]:
with strategy.scope():
    replicated_model = tf.keras.models.load_model(path)
    replicated_model.compile(loss='sparse_categorical_crossentropy',
                            optimizer=tf.keras.optimizers.Adam(),
                            metrics=['accuracy'])

    eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
    print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

    157/Unknown - 3s 21ms/step - loss: 0.0439 - accuracy: 0.9901Eval loss: 0.04394236892377211, Eval Accuracy: 0.9901000261306763
